# Prompt Cache Lab - 문서 기반 RAG

# 사전 설정

In [1]:
%load_ext autoreload
%autoreload 2
%pip install ipywidgets
%pip install boto3 botocore
%pip install pandas

Looking in indexes: https://pypi.org/simple, https://plugin.us-east-1.prod.workshops.aws
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://plugin.us-east-1.prod.workshops.aws
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://plugin.us-east-1.prod.workshops.aws
Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3, botocore
retry_config = botocore.config.Config(
    retries={"max_attempts": 1, "mode": "standard"}
)
session = boto3.Session(
    region_name='us-west-2'
)

bedrock_client = session.client("bedrock-runtime", config=retry_config)

print ("\n== FM lists ==")
model_list = session.client("bedrock").list_foundation_models()['modelSummaries']
print('\n'.join([model['modelId'] for model in model_list if model['modelId'].startswith('anth')]))


== FM lists ==
anthropic.claude-3-5-sonnet-20241022-v2:0:18k
anthropic.claude-3-5-sonnet-20241022-v2:0:51k
anthropic.claude-3-5-sonnet-20241022-v2:0:200k
anthropic.claude-3-5-sonnet-20241022-v2:0
anthropic.claude-3-7-sonnet-20250219-v1:0
anthropic.claude-3-5-haiku-20241022-v1:0
anthropic.claude-instant-v1:2:100k
anthropic.claude-instant-v1
anthropic.claude-v2:0:18k
anthropic.claude-v2:0:100k
anthropic.claude-v2:1:18k
anthropic.claude-v2:1:200k
anthropic.claude-v2:1
anthropic.claude-v2
anthropic.claude-3-sonnet-20240229-v1:0:28k
anthropic.claude-3-sonnet-20240229-v1:0:200k
anthropic.claude-3-sonnet-20240229-v1:0
anthropic.claude-3-haiku-20240307-v1:0:48k
anthropic.claude-3-haiku-20240307-v1:0:200k
anthropic.claude-3-haiku-20240307-v1:0
anthropic.claude-3-opus-20240229-v1:0:12k
anthropic.claude-3-opus-20240229-v1:0:28k
anthropic.claude-3-opus-20240229-v1:0:200k
anthropic.claude-3-opus-20240229-v1:0
anthropic.claude-3-5-sonnet-20240620-v1:0:18k
anthropic.claude-3-5-sonnet-20240620-v1:0:5

### Converse API 활용 방법 소개

In [3]:
model_id = 'anthropic.claude-3-5-haiku-20241022-v1:0'
messages = [{
    'role': 'user',
    'content': [{'text': f"아마존 베드락은 무엇인가요?"}]
}]

result = bedrock_client.converse(
    modelId=model_id,
    messages=messages,
    system=[{'text': "당신은 AWS의 솔루션즈 아키텍트입니다."}]
)

print(result['output']['message']['content'][0]['text'])

아마존 베드락(Amazon Bedrock)은 AWS에서 제공하는 생성형 AI 서비스입니다. 주요 특징은 다음과 같습니다:

1. 핵심 기능
- 대규모 언어 모델(LLM) 접근
- 다양한 AI 모델 통합
- 완전 관리형 서비스

2. 지원 모델
- Anthropic의 Claude
- AI21 Labs의 Jurrassic
- Stability AI의 이미지 생성 모델
- Amazon의 자체 모델

3. 주요 장점
- 쉬운 모델 통합
- 안전하고 프라이버시 보호
- 엔터프라이즈 수준의 보안
- 빠른 AI 애플리케이션 개발

4. 사용 사례
- 챗봇 개발
- 텍스트 생성
- 요약
- 번역
- 고객 서비스 자동화


### Prompt Cache 없이 문서 기반 RAG 수행

In [4]:
def converse_with_document(system_prompt, document, user_query, model_id):
    # 문서 정의
    document_content =  f"## document:\n{document} "
    
    # System Prompt 정의
    system_list = [
        {
            "text": system_prompt
        }
    ]
    
    # Conversation 추가 정의
    message_list = [
        {
            'role': 'user',
            'content': [
                {
                    'text': document_content
                },
                {
                    'text': user_query
                },
            ]
        },
    ]

    # 추론에 필요한 Hyperparameter 정의
    inference_config = {
        'maxTokens': 4096,
        'temperature': 0,
        'topP': 1
    }

    # Converse API 호출
    response = bedrock_client.converse(
        system=system_list,
        messages=message_list,
        modelId=model_id,
        inferenceConfig=inference_config
    )
    
    print (response["output"]["message"]["content"][0]["text"])
    print (response['usage'])
    return (response['metrics']['latencyMs'], response['usage'].get('cacheReadInputTokens', 0), response['usage'].get('cacheWriteInputTokens', 0), response['usage']['inputTokens'], response['usage']['outputTokens'])


#### Prompt Cache 없이 사용되는 토큰량/Response Time 비교 (3번의 질문)

In [5]:
usage_history = []  # 결과저장 list 

In [6]:
# Argument 정의
# Document
file_path = "documents/romeo_and_juliet_korean.pdf"

import pdfplumber
import re

full_text = ""

with pdfplumber.open(file_path) as pdf:
    total_pages = len(pdf.pages)
    # end_page = total_pages
    end_page = 20
    
    for page_num in range(0, end_page):
        text = pdf.pages[page_num].extract_text()
        text = re.sub(r'\s+', ' ', text).strip()
        full_text = "".join([full_text, text, " "])

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

In [7]:
len(full_text)

19369

In [8]:
model_id = 'anthropic.claude-3-5-haiku-20241022-v1:0'

system_prompt = "당신은 주어진 대본 속 주인공의 역할을 이해한 해설가 입니다. 모든 내용은 주어진 대본을 바탕으로 답변하세요."
document = full_text
questions = [
    "주어진 대본은 어떤 내용을 담고 있나요?",
    "남녀 주인공은 각각 어떤 상황인가요?",
    "이 소설과 관련된 퀴즈를 만들려고 합니다. 3개 정도의 질문 리스트를 만들어 주세요."
]

In [9]:
for question in questions:
    result = converse_with_document(system_prompt, document, question, model_id)
    usage_history.append(result)

이 대본은 셰익스피어의 유명한 비극 "로미오와 줄리엣"의 1막 부분입니다. 주요 내용은 다음과 같습니다:

1. 프롤로그에서 베로나의 두 원수 집안(몬타규와 캐풀렛)의 오랜 갈등과 그 자녀들의 비극적 사랑 이야기를 예고합니다.

2. 1막 1장에서는 캐풀렛과 몬타규 집안의 하인들 사이의 싸움이 시작되고, 이로 인해 양 집안 사람들이 서로 대립하게 됩니다.

3. 1막 2장에서는 캐풀렛이 패리스 백작에게 딸 줄리엣과의 결혼을 고려하고 있음을 보여줍니다.

4. 1막 3장에서는 줄리엣의 어머니와 유모가 줄리엣에게 결혼에 대해 이야기합니다.

5. 1막 4장에서 로미오와 그의 친구들이 캐풀렛 집의 가면무도회에 참석합니다.

6. 1막 5장이 가장 중요한 장면으로, 로미오와 줄리엣이 처음으로 만나 서로에게 매료되고 키스를 나누는 장면입니다. 이들은 서로 원수 집안의 자녀임을 알게 됩니다.

이 장면은 두 주인공의 운명적인 사랑의 시작을 보여주며, 그들의 사랑이 얼마나 금지된 것인지를 동시에 드러냅니다.
{'inputTokens': 21502, 'outputTokens': 546, 'totalTokens': 22048}
이 대본의 1막 5장에서 로미오와 줄리엣의 상황은 다음과 같습니다:

로미오:
- 캐풀렛 집의 가면무도회에 몬타규 가문의 일원으로 참석
- 처음에는 로잘라인을 사랑하고 있었으나, 줄리엣을 보고 즉시 그녀에게 매료됨
- 줄리엣과 만나 성자와 순례자의 비유를 사용하며 서로 키스를 나눔
- 줄리엣이 캐풀렛 집 딸임을 알고 "내 목숨은 원수의 저당물이 됐구나"라고 말함

줄리엣:
- 캐풀렛 집의 딸로, 아직 열네 살의 어린 나이
- 가면무도회에 참석 중 로미오를 만나 즉시 그에게 매력을 느낌
- 로미오와 키스를 나누며 서로에 대한 감정을 표현
- 로미오가 몬타규 가문의 일원임을 알고 "단 하나의 내 순정이 단 하나의 내 증오에서 싹트다니!"라고 말하며 복잡한 감정을 드러냄

두 사람 모두 서로에 대한 강렬한 사랑의 감정을 느끼면서도, 서로 원수 집안의 자녀라는

In [10]:
import pandas as pd 

# 비교용 데이터 프레임 정의
cols =[ "LatencyMs", "CacheRead", "CacheWrite", "Inputs", "Outputs"]
df = pd.DataFrame(usage_history)
df.columns = cols
df

,LatencyMs,CacheRead,CacheWrite,Inputs,Outputs
0,12818,0,0,21502,546
1,9866,0,0,21501,493
2,11345,0,0,21529,574


### Prompt Cache 활용하여 문서 기반 RAG 수행

In [11]:
usage_history = []

In [12]:
def converse_with_cached_document(system_prompt, document, user_query, model_id):
    # 문서 정의
    document_content =  f"## document:\n{document} "
    
    # System Prompt 정의
    system_list = [
        {
            "text": system_prompt
        },
        {
            "cachePoint": {
                "type": "default"
            }
        }
    ]    
    # Conversation 추가 정의
    message_list = [
        {
            'role': 'user',
            'content': [
                {
                    'text': document_content
                },
                {
                    "cachePoint": {
                        "type": "default"
                    }
                },
                {
                    'text': user_query
                },
            ]
        },
    ]

    # 추론에 필요한 Hyperparameter 정의
    inference_config = {
        'maxTokens': 4096,
        'temperature': 0,
        'topP': 1
    }

    # Converse API 호출
    response = bedrock_client.converse(
        system=system_list,
        messages=message_list,
        modelId=model_id,
        inferenceConfig=inference_config
    )

    print (response["output"]["message"]["content"][0]["text"])
    return (response['metrics']['latencyMs'], response['usage'].get('cacheReadInputTokens', 0), response['usage'].get('cacheWriteInputTokens', 0), response['usage']['inputTokens'], response['usage']['outputTokens'])


#### Prompt Cache 활용 시 사용되는 토큰량/Response Time 비교 (3번의 질문)

In [14]:
for question in questions:
    result = converse_with_cached_document(system_prompt, document, question, model_id)
    usage_history.append(result)

이 대본은 셰익스피어의 유명한 비극 "로미오와 줄리엣"의 1막 부분입니다. 주요 내용은 다음과 같습니다:

1. 프롤로그에서 베로나의 두 원수 집안(몬타규와 캐풀렛)의 오랜 갈등과 그 자녀들의 비극적 사랑 이야기를 예고합니다.

2. 1막 1장에서는 캐풀렛과 몬타규 집안의 하인들 사이의 싸움이 시작되고, 이로 인해 양 집안 사람들이 서로 대립하게 됩니다.

3. 1막 2장에서는 캐풀렛이 패리스 백작에게 딸 줄리엣과의 결혼을 고려하고 있음을 보여줍니다.

4. 1막 3장에서는 줄리엣의 어머니와 유모가 줄리엣에게 결혼에 대해 이야기합니다.

5. 1막 4장에서 로미오와 그의 친구들이 캐풀렛 집의 가면무도회에 참석합니다.

6. 1막 5장이 가장 중요한 장면으로, 로미오와 줄리엣이 처음으로 만나 서로에게 매료되고 키스를 나누는 장면입니다. 이들은 서로 원수 집안의 자녀임을 알게 됩니다.

이 장면은 두 주인공의 운명적인 사랑의 시작을 보여주며, 그들의 사랑이 얼마나 금지된 것인지를 동시에 드러냅니다.
이 대본의 1막 5장에서 로미오와 줄리엣의 상황은 다음과 같습니다:

로미오:
- 캐풀렛 집의 가면무도회에 몬타규 가문의 일원으로 참석
- 처음에는 로잘라인을 사랑하고 있었으나, 줄리엣을 보고 즉시 그녀에게 매료됨
- 줄리엣과 만나 성자와 순례자의 비유를 사용하며 서로 키스를 나눔
- 줄리엣이 캐풀렛 집 딸임을 알고 "내 목숨은 원수의 저당물이 됐구나"라고 말함

줄리엣:
- 캐풀렛 집의 딸로, 아직 열네 살의 어린 나이
- 가면무도회에 참석 중 로미오를 만나 즉시 그에게 매력을 느낌
- 로미오와 키스를 나누며 서로에 대한 감정을 표현
- 로미오가 몬타규 가문의 일원임을 알고 "단 하나의 내 순정이 단 하나의 내 증오에서 싹트다니!"라고 말하며 복잡한 감정을 드러냄

두 사람 모두 서로에 대한 강렬한 사랑의 감정을 느끼면서도, 서로 원수 집안의 자녀라는 사실에 대한 갈등을 동시에 경험하고 있습니다.
로미오와 줄리엣 대본을 바탕으로 다음과 같은 퀴즈 문제들을 제안드립니다

In [15]:
cols =[ "LatencyMs", "CacheRead", "CacheWrite", "Inputs", "Outputs"]
df = pd.DataFrame(usage_history)
df.columns = cols
df

,LatencyMs,CacheRead,CacheWrite,Inputs,Outputs
0,11177,0,21475,27,546
1,8845,21475,0,26,493
2,10193,21475,0,54,574
